In [7]:
import pandas as pd
import numpy as np
from scipy import spatial
from sklearn.decomposition import PCA
from tqdm.auto import tqdm

In [2]:
word2vec_embeddings = pd.read_parquet('/pio/scratch/1/recommender_systems/processed/word2vec/amazon-clothes/5-core/item_item_embeddings.parquet')

In [5]:
word2vec_embeddings

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
index,,,,,,,,,,,,,,,,,,,,,
B00SU52460,-0.078735,0.031405,0.057666,0.090253,0.055262,0.004454,0.068138,0.140214,-0.160675,-0.001144,...,-0.028228,-0.120070,-0.141606,-0.066599,0.224218,-0.110949,0.103306,-0.067999,0.113143,0.087766
B00STBRT9M,-0.068344,0.023045,0.061581,0.102385,0.074816,0.014265,0.068359,0.128264,-0.166192,-0.011253,...,-0.026627,-0.105604,-0.149194,-0.069288,0.214240,-0.120073,0.093751,-0.074356,0.106935,0.087801
B00SU5244M,-0.075733,0.023512,0.059046,0.085086,0.056547,0.011237,0.074188,0.118348,-0.144083,-0.033818,...,-0.030343,-0.120872,-0.117350,-0.074961,0.228749,-0.112058,0.121153,-0.060023,0.108016,0.094660
B00WRGPHY4,-0.072343,0.012610,0.053967,0.094973,0.055312,0.006420,0.070272,0.129301,-0.158411,-0.018321,...,-0.036316,-0.110264,-0.128595,-0.070194,0.231627,-0.117734,0.111027,-0.080635,0.110474,0.094160
B00V6K7P0G,-0.063687,0.027942,0.054714,0.093803,0.067540,-0.003258,0.080216,0.139799,-0.150908,-0.019819,...,-0.017860,-0.111070,-0.136759,-0.065150,0.219500,-0.126879,0.110436,-0.077520,0.109715,0.083321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B00EVWCKGK,-0.009977,0.114948,0.026678,0.038227,0.067839,-0.124184,0.146636,0.247998,-0.134573,-0.112451,...,0.162464,0.008654,-0.039211,-0.000062,0.206180,-0.016075,0.065916,-0.071067,0.019635,0.025326
B019T96K7Y,-0.012741,0.075559,0.031464,0.040446,0.022506,-0.156780,0.026824,0.282930,-0.168034,-0.121483,...,0.157675,-0.015005,-0.056732,-0.096309,0.183719,-0.005037,0.041290,-0.012217,0.058867,0.020988
B00Q1O4BQI,0.004234,0.083226,0.014779,0.080230,0.023185,-0.120007,0.155374,0.236055,-0.146596,-0.120077,...,0.122532,-0.004304,-0.033180,-0.010378,0.234828,-0.030729,0.090226,-0.060556,0.000169,0.009621


In [10]:
pca = PCA(n_components=2)
pca.fit(word2vec_embeddings)

PCA(n_components=2)

In [4]:
def analyse_ABX(path, items_embeddings, pca):
    abx_tests = pd.read_json(path, lines=True)
    A = np.zeros((10000, 100))
    B = np.zeros((10000, 100))
    X = np.zeros((10000, 100))

    for i, test in abx_tests.iterrows():
        A[i, :] = items_embeddings.loc[test["A"]]
        B[i, :] = items_embeddings.loc[test["B"]]
        X[i, :] = items_embeddings.loc[test["X"]]

    dist_A = ((A - X)**2).sum(axis=1)
    dist_B = ((B - X)**2).sum(axis=1)

    cos_dist_A = np.zeros(10000)
    cos_dist_B = np.zeros(10000)

    for i in range(10000):
        cos_dist_A[i] = spatial.distance.cosine(A[i, :], X[i, :])
        cos_dist_B[i] = spatial.distance.cosine(B[i, :], X[i, :])

    if pca is not None:
        pca_A = pca.transform(A)
        pca_B = pca.transform(B)
        pca_X = pca.transform(X)

        dist_pca_A = ((pca_A - pca_X)**2).sum(axis=1)
        dist_pca_B  = ((pca_B - pca_X)**2).sum(axis=1)

        cos_dist_pca_A = np.zeros(10000)
        cos_dist_pca_B = np.zeros(10000)

        for i in range(10000):
            cos_dist_pca_A[i] = spatial.distance.cosine(pca_A[i, :], pca_X[i, :])
            cos_dist_pca_B[i] = spatial.distance.cosine(pca_B[i, :], pca_X[i, :])

    return [(dist_A < dist_B).mean(), (dist_pca_A < dist_pca_B).mean(), ((dist_A < dist_B) == (dist_pca_A < dist_pca_B)).mean()], [(cos_dist_A < cos_dist_B).mean(), (cos_dist_pca_A < cos_dist_pca_B).mean(), ((cos_dist_A < cos_dist_B) == (cos_dist_pca_A < cos_dist_pca_B)).mean()]

In [11]:
analyse_ABX('/pio/scratch/1/recommender_systems/interim/ABX_tests/5_core.json', word2vec_embeddings, pca)

([0.6349, 0.6428, 0.8697], [0.6349, 0.6496, 0.7293])